In [93]:
from PIL import Image
import numpy as np
import torch
from torchvision import datasets, transforms, models

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [117]:
#for self test

filepath1 = 'flowers/test/1/image_06743.jpg'
image1 = Image.open(filepath1)

print('image1 format: ',image1.format )
print('image1 size: ',image1.size )
print('image1 mode: ',image1.mode )


image1 format:  JPEG
image1 size:  (500, 601)
image1 mode:  RGB


In [118]:
size = (256,256)
image1 = image1.resize(size)

In [119]:
print('image1 format: ',image1.format )
print('image1 size: ',image1.size )
print('image1 mode: ',image1.mode )

image1 format:  None
image1 size:  (256, 256)
image1 mode:  RGB


In [120]:
width, height = image1.size
print('width: ', width)
print('height: ', height)
new_width = 224
new_height =224
left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2
print('left: ',left)
print('top: ',top)
print('right: ',right)
print('bottom: ',bottom)
image1 = image1.crop((left, top, right, bottom))

width:  256
height:  256
left:  16.0
top:  16.0
right:  240.0
bottom:  240.0


In [121]:
print('image1 format: ',image1.format )
print('image1 size: ',image1.size )
print('image1 mode: ',image1.mode )

image1 format:  None
image1 size:  (224, 224)
image1 mode:  RGB


In [122]:
np_image1 = np.array(image1)

In [123]:
image1 = np_image1/255
image1

array([[[ 0.02352941,  0.05098039,  0.02352941],
        [ 0.05098039,  0.07843137,  0.05098039],
        [ 0.09019608,  0.10980392,  0.08627451],
        ..., 
        [ 0.12941176,  0.18823529,  0.09803922],
        [ 0.22745098,  0.30588235,  0.2       ],
        [ 0.23921569,  0.31764706,  0.21960784]],

       [[ 0.06666667,  0.09411765,  0.0627451 ],
        [ 0.08235294,  0.10980392,  0.07843137],
        [ 0.08235294,  0.10196078,  0.0745098 ],
        ..., 
        [ 0.04705882,  0.0745098 ,  0.04313725],
        [ 0.02352941,  0.07843137,  0.01176471],
        [ 0.05490196,  0.10980392,  0.04313725]],

       [[ 0.09803922,  0.1254902 ,  0.09411765],
        [ 0.09019608,  0.11764706,  0.08627451],
        [ 0.0627451 ,  0.08235294,  0.05490196],
        ..., 
        [ 0.05490196,  0.07058824,  0.0745098 ],
        [ 0.07843137,  0.11764706,  0.0745098 ],
        [ 0.10980392,  0.15686275,  0.10196078]],

       ..., 
       [[ 0.18431373,  0.24705882,  0.10588235],
        

In [125]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
#transform numpy array to torch tensor
image1_normalized = (image1 - mean) / std

In [126]:
image1_normalized

array([[[-2.01515541, -1.80812325, -1.69986928],
        [-1.89528213, -1.68557423, -1.57786492],
        [-1.72403459, -1.54551821, -1.42100218],
        ..., 
        [-1.55278705, -1.19537815, -1.3687146 ],
        [-1.12466821, -0.67016807, -0.91555556],
        [-1.07329395, -0.61764706, -0.82840959]],

       [[-1.82678311, -1.61554622, -1.52557734],
        [-1.7582841 , -1.54551821, -1.45586057],
        [-1.7582841 , -1.58053221, -1.47328976],
        ..., 
        [-1.91240688, -1.70308123, -1.61272331],
        [-2.01515541, -1.68557423, -1.75215686],
        [-1.87815738, -1.54551821, -1.61272331]],

       [[-1.68978508, -1.4754902 , -1.38614379],
        [-1.72403459, -1.5105042 , -1.42100218],
        [-1.84390787, -1.66806723, -1.56043573],
        ..., 
        [-1.87815738, -1.72058824, -1.47328976],
        [-1.77540885, -1.5105042 , -1.47328976],
        [-1.63841082, -1.33543417, -1.3512854 ]],

       ..., 
       [[-1.3130405 , -0.93277311, -1.33385621],
        